#Genetic algorithm in Python


In [ ]:
import numpy as np, matplotlib.pyplot as plt
from matplotlib.pyplot import figure

class Population():
    '''Une classe où nous pouvons stocker la population,
    le score de chaque individu, les parents sélectionnés pour le crossover,
    l'individu avec le meilleur score et la matrice des distances.'''
    def __init__(self,villes, mat_distance,nbr_population):
        # initialiser la première population de façon aléatoire  
        self.sac = np.asarray([np.random.permutation(villes) for _ in range(nbr_population)])
        self.parents = [] # une liste vide pour stocker les parents
        self.score = 0 # une variable pour stocker le score des individus 
        self.meilleur = None # une variable pour stocker le meilleur individu
        self.mat_distance = mat_distance # une variable pour stocker la matrice des distances


    def fitness(self, chromosome):
        ''''Cette fonction calcule le score des individus.'''
        return sum(
            [
                self.mat_distance[chromosome[i], chromosome[i + 1]]
                for i in range(len(chromosome) - 1)
            ]
        )+self.mat_distance[chromosome[0], chromosome[len(chromosome) - 1]]

    def evaluate(self):
        '''Cette fonction évalue les individus et attribue une probabilité
        à chacun d'être sélectionné comme parent.  '''
        distances = np.asarray(
            [self.fitness(chromosome) for chromosome in self.sac]
        )

        self.score = np.min(distances) # stocker le meilleur score d'une generation.
        self.meilleur = self.sac[distances.tolist().index(self.score)] # stocker l'individu avec le meilleur score.
        self.parents.append(self.meilleur) # selectionneioné l'individu avec le meilleur score comme un parent.
        if False in (distances[0] == distances):
            # on recherche le score le plus élevé et nous soustrayons de celui-ci les valeurs des autres scores,
            # et nous stockons les résultats dans le même tableau en gardant les mêmes indices.
            distances = np.max(distances) - distances
        return distances / np.sum(distances) #ceci renvoie un tableau de probabilités

    def selectionne(self, nbr_parents):
        '''Cette fonction sélectionne les parents pour l'étape de crossover'''
        fit = self.evaluate() #On evalué la generation
        while len(self.parents) < nbr_parents: #k représent le nombre de parents a selectioné
            indice = np.random.randint(0, len(fit)) #on choisit un individu aléatoirement
            # on génère un nombre aléatoire entre 0 et 1 
            # et on le compare à la probabilité de l'individu d'être choisi ou non 
            if fit[indice] > np.random.rand(): 
                self.parents.append(self.sac[indice])
        self.parents = np.asarray(self.parents) #nous convertissons la liste des parents en tableau de numpy
    def crossover(self, p_cross):
        ''''Cette fonction effectue le crossover où elle sélectionne
        deux parents et en fonction de la probabilité.'''
        enfants = []
        count, taille = self.parents.shape #On stocke la dimension du tableau des parents 
        for _ in range(len(self.sac)):
            # on génère un nombre aléatoire entre 0 et 1 et on le compare à la probabilité de crossover 
            # si le nombre est inférieur à la probabilité on effectue le crossover.
            if np.random.rand() > p_cross:
                # Si l'individu n'a pas été choisi, 
                # on le stock dans la liste des enfants pour la prochaine génération.  
                enfants.append(
                    list(self.parents[np.random.randint(count, size=1)[0]])
                )
            else:
                parent_1, parent_2 = self.parents[
                    np.random.randint(count, size=2), :
                ]
                # on génère deux nombres entiers aléatoires entre 0 et le nombre de gènes dans un individu.
                indice = np.random.choice(range(taille), size=2, replace=False)
                # nous stockons les deux nombres générés dans deux variables,
                # le plus petit nombre dans 'debut' et l'autre nombre dans 'fin'.
                debut, fin = min(indice), max(indice) 
                enfant = [None] * taille # on crée une liste enfant dont les valeurs sont 'None'.
                for i in range(debut, fin + 1, 1):
                    # on remplit l'enfant avec les gènes du premier père en gardant les mêmes indices, 
                    # on commence à l'indice 'debut' et on s'arrête à 'fin'.
                    enfant[i] = parent_1[i]
                pointeur = 0 
                for i in range(taille):
                    # maintenant, on remplit le reste avec les gènes manquants du deuxième parent dans le même ordre.
                    if enfant[i] is None:
                        while parent_2[pointeur] in enfant:
                            pointeur += 1
                        enfant[i] = parent_2[pointeur]
                enfants.append(enfant)
        return enfants


    def mutate(self, p_cross, p_mut):
        def swap(chromosome):
            '''Cette fonction permute les indices de deux villes dans un individu.'''
            # on génère deux nombres entiers aléatoires entre 0 et le nombre de gènes dans un individu.
            a, b = np.random.choice(len(chromosome), 2)
            # on change les indices des gènes dans l'individu  
            (chromosome[a], chromosome[b]) = (chromosome[b], chromosome[a])
            return chromosome
        '''Cette fonction exécute la mutation en utilisant la fonction swap.'''
        next_sac = [] # on crée une liste vide pour stocker la génération suivante. 
        enfants = self.crossover(p_cross) # on effectue le crossover sur la génération actuelle.
        for enfant in enfants:
            # on génère un nombre aléatoire entre 0 et 1 et on le compare à la probabilité de mutation 
            # si le nombre est inférieur à la probabilité on effectue le mutation.
            if np.random.rand() < p_mut:
                # si la condition est stratifiée on effectue la mutation   
                next_sac.append(swap(enfant))
            else:
                next_sac.append(enfant)
        self.parents=self.parents.tolist()        
        return next_sac


def genetic_algorithm(
    # les entrées de l'algorithme.
    villes, # La liste des villes.
    mat_distance, # La mtrice des disctances. 
    nbr_population, # Le nombre de popultaion.
    nbr_iteration, # Le nombre d'iteration (génération). 
    selectivity=0.2, # on sélectionne 20% de la population comme parents.
    p_cross=0.7, # La probability de croisement
    p_mut=0.1, # la probability de mutation.
):
    pop = Population(villes, mat_distance, nbr_population) # Initialisation de la population.
    meilleur = pop.meilleur # Le variable où on stocke le meilleur individu. 
    score = float("inf") # On initialise la variable score avec une valeur infinie.
    histoire = [] # La liste où on stocke le score du meilleur individu de chaque génération.
    for i in range(nbr_iteration): # La boucle où l'on exécute l'algorithme.
        # Ici, on sélectionne les individus qui seront les parents et le nombre de parents en entrée.
        pop.selectionne(nbr_population * selectivity)
        histoire.append(pop.score) # on ajoute le score du meilleur individu de la génération à la liste.
        # ici, on compare le meilleur score de la génération actuelle avec le meilleur score actuel de tous les temps 
        # et on le stocke s'il est meilleur. 
        if pop.score < score:
            meilleur = pop.meilleur
            score = pop.score
        enfants = pop.mutate(p_cross, p_mut) #on effectue les opérations de croisement et de mutation.
        pop.sac = enfants # On prend les enfants de la génération actuelle comme individus pour la suivante.  

    # Les commandes pour créer le graphe.
    figure(figsize=(8, 6), dpi=100)   
    plt.plot(range(len(histoire)), histoire, color="blue",linewidth=1.0)
    plt.xlabel("Géneration")
    plt.ylabel("Score")
    plt.show()  
    return score, meilleur

In [ ]:
from PyQt5 import QtCore, QtGui, QtWidgets


class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(612, 524)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.verticalLayout_2 = QtWidgets.QVBoxLayout(self.centralwidget)
        self.verticalLayout_2.setObjectName("verticalLayout_2")
        self.verticalLayout = QtWidgets.QVBoxLayout()
        self.verticalLayout.setObjectName("verticalLayout")
        self.nbr_villes = QtWidgets.QLineEdit(self.centralwidget)
        self.nbr_villes.setEnabled(True)
        self.nbr_villes.setText("")
        self.nbr_villes.setObjectName("nbr_villes")
        self.nbr_villes.setStyleSheet("border: 1px solid black;")
        self.verticalLayout.addWidget(self.nbr_villes)
        self.horizontalLayout = QtWidgets.QHBoxLayout()
        self.horizontalLayout.setObjectName("horizontalLayout")
        self.nbr_population = QtWidgets.QLineEdit(self.centralwidget)
        self.nbr_population.setObjectName("nbr_population")
        self.nbr_population.setStyleSheet("border: 1px solid black;")
        self.horizontalLayout.addWidget(self.nbr_population)
        self.nbr_iterations = QtWidgets.QLineEdit(self.centralwidget)
        self.nbr_iterations.setObjectName("nbr_iterations")
        self.nbr_iterations.setStyleSheet("border: 1px solid black;")
        self.horizontalLayout.addWidget(self.nbr_iterations)
        self.verticalLayout.addLayout(self.horizontalLayout)
        self.pushButton_matrix = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton_matrix.setObjectName("pushButton_matrix")
        self.verticalLayout.addWidget(self.pushButton_matrix)
        self.Random_matrix = QtWidgets.QPushButton(self.centralwidget)
        self.Random_matrix.setObjectName("Random_matrix")
        self.verticalLayout.addWidget(self.Random_matrix)
        self.tableWidget = QtWidgets.QTableWidget(self.centralwidget)
        self.tableWidget.setObjectName("tableWidget")
        self.tableWidget.setColumnCount(0)
        self.tableWidget.setRowCount(0)
        self.tableWidget.horizontalHeader().setVisible(True)
        self.tableWidget.verticalHeader().setVisible(True)
        self.tableWidget.setStyleSheet("border: 1px solid black;")
        self.verticalLayout.addWidget(self.tableWidget)
        self.auto_complete = QtWidgets.QPushButton(self.centralwidget)
        self.auto_complete.setObjectName("auto_complete")
        self.verticalLayout.addWidget(self.auto_complete)
        self.Get_matrix_content = QtWidgets.QPushButton(self.centralwidget)
        self.Get_matrix_content.setObjectName("Get_matrix_content")
        self.verticalLayout.addWidget(self.Get_matrix_content)
        self.verticalLayout_2.addLayout(self.verticalLayout)
        self.affichage = QtWidgets.QTextBrowser(self.centralwidget)
        self.affichage.setEnabled(True)
        sizePolicy = QtWidgets.QSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Fixed)
        sizePolicy.setHorizontalStretch(0)
        sizePolicy.setVerticalStretch(0)
        sizePolicy.setHeightForWidth(self.affichage.sizePolicy().hasHeightForWidth())
        self.affichage.setSizePolicy(sizePolicy)
        self.affichage.setMaximumSize(QtCore.QSize(16777215, 100))
        self.affichage.setBaseSize(QtCore.QSize(0, 0))
        font = QtGui.QFont()
        font.setPointSize(22)
        self.verticalLayout_2.addWidget(self.affichage)
        MainWindow.setCentralWidget(self.centralwidget)
        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "Algorithme Génétique (TSP)"))
        self.nbr_villes.setPlaceholderText(_translate("MainWindow", "Nombre de villes"))
        self.nbr_population.setPlaceholderText(_translate("MainWindow", "Nombre de population"))
        self.nbr_iterations.setPlaceholderText(_translate("MainWindow", "Nombre de generation"))
        self.pushButton_matrix.setText(_translate("MainWindow", "Genéré une matrice vide"))
        self.Random_matrix.setText(_translate("MainWindow", "Genéré une matrice aléatoire"))
        self.auto_complete.setText(_translate("MainWindow", "Remplir le reste de la matrice"))
        self.Get_matrix_content.setText(_translate("MainWindow", "Résoudre"))




In [ ]:
from PyQt5.QtWidgets import (
    QMainWindow,
    QWidget,
    QApplication,
    QTableWidgetItem
)

import sys, numpy as np
from timeit import default_timer

class MainWindow(QMainWindow):
    def __init__(self):
        QWidget.__init__(self)
        self.ui = Ui_MainWindow()
        self.ui.setupUi(self)
        self.show()
        self.buttons_actions()

        
    def buttons_actions(self):
        self.ui.pushButton_matrix.clicked.connect(
            self.generateMatrix
        )
        self.ui.Get_matrix_content.clicked.connect(
            self.getMatrixContent
        )
        self.ui.Random_matrix.clicked.connect(
            self.generate_random_matrix
        )
        self.ui.auto_complete.clicked.connect(
            self.auto_complete
        )


    def generateMatrix(self):
        number_of_line = int(self.ui.nbr_villes.text())
        
        for j in range(0,number_of_line):
            self.ui.tableWidget.insertColumn(j)

        for i in range(0,number_of_line):
            self.ui.tableWidget.insertRow(i)
            for j in range(0, number_of_line):
                self.ui.tableWidget.setItem(i , j, QTableWidgetItem(""))
                if i == j:
                    self.ui.tableWidget.setItem(i , j, QTableWidgetItem("0"))


    def generate_random_matrix(self): 
        number_of_line = int(self.ui.nbr_villes.text())

        for j in range(0,number_of_line):
            self.ui.tableWidget.insertColumn(j)

        for i in range(0,number_of_line):
            self.ui.tableWidget.insertRow(i)
            for j in range(0, number_of_line):
                random_number=str(np.random.randint(1,100))
                self.ui.tableWidget.setItem(i , j, QTableWidgetItem(random_number))
                self.ui.tableWidget.setItem(j , i, QTableWidgetItem(random_number))
                if i == j:
                    self.ui.tableWidget.setItem(i , j, QTableWidgetItem("0"))


    def auto_complete(self):
        count_row = self.ui.tableWidget.columnCount()
        x=1
        if self.ui.tableWidget.item(0,1).text() == "":
            for k in range (1,count_row):
                for l in range(x):
                    item = self.ui.tableWidget.item(k,l).text()
                    self.ui.tableWidget.setItem(l , k, QTableWidgetItem(item))
                x+=1           
        else:
            for k in range (1,count_row):
                for l in range(x):
                    item = self.ui.tableWidget.item(l,k).text()
                    self.ui.tableWidget.setItem(k , l, QTableWidgetItem(item))
                x+=1           


    def getMatrixContent(self):
        count_row = self.ui.tableWidget.columnCount()
        mat_distance = []
        for i in range(0,count_row):
            data=[]
            for j in range(0,count_row):
                data.append(
                    float(
                        self.ui.tableWidget.item(i,j).text()
                    )
                )
            mat_distance.append(data)    
        mat_distance=np.array(mat_distance)


        nbr_population = int(self.ui.nbr_population.text())
        nbr_iteration = int(self.ui.nbr_iterations.text())
        nbr_villes=int(self.ui.nbr_villes.text())
        villes=range(nbr_villes)


        debut=default_timer()
        (score,meilleur)= genetic_algorithm(villes,mat_distance,nbr_population,nbr_iteration)
        fin=default_timer()
        self.ui.affichage.setText(f"La meilleur solution trouvé est {meilleur} avec un score de {score}.\n\nLe temps d'execution est {fin-debut}s.")



if __name__ == "__main__":
    import sys
    app = QApplication(sys.argv)
    window = MainWindow()
    sys.exit(app.exec_())
    